# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Name of the CSV file
file = 'output_data/cities.csv'

# The correct encoding must be used to read the CSV in pandas
city_data_df = pd.read_csv(file, encoding="ISO-8859-1")
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Alfred,-33.59,26.89,63.00,82,41,4.00,ZA,1592321375
1,Torbay,47.67,-52.73,66.99,55,20,11.41,CA,1592321375
2,Srednekolymsk,67.45,153.68,70.03,43,65,6.82,RU,1592321380
3,Kavaratti,10.57,72.64,84.81,78,100,18.66,IN,1592321456
4,Barrow,71.29,-156.79,35.60,80,75,12.75,US,1592321194
...,...,...,...,...,...,...,...,...,...
558,Kikwit,-5.04,18.82,85.39,35,0,2.77,CD,1592321527
559,Aklavik,68.22,-135.01,57.20,82,90,3.36,CA,1592321527
560,Angoche,-16.23,39.91,70.84,84,99,12.33,MZ,1592321527
561,Andahuaylas,-13.66,-73.39,61.32,48,29,5.82,PE,1592321527


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 # Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into locations 
locations = city_data_df[['Lat', 'Lng']].astype(float)

# Store Humidity as the weight
humidity = city_data_df['Humidity'].astype(float)

In [5]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
max_indices = city_data_df.index[city_data_df['Humidity'] > 79].tolist()
min_indices = city_data_df.index[city_data_df['Humidity'] < 71].tolist()
indices = max_indices + min_indices
narrow_city_data = city_data_df.drop(indices, inplace=False)

wind_indices = narrow_city_data.index[narrow_city_data['Wind Speed'] > 9].tolist()
narrower_city_data = narrow_city_data.drop(wind_indices, inplace=False)

cloud_indices = narrower_city_data.index[narrower_city_data['Cloudiness'] > 0].tolist()
narrowest_city_data = narrower_city_data.drop(cloud_indices, inplace=False)

narrowest_city_data = narrowest_city_data.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrowest_city_data.copy()
hotel_df['Hotel Name'] = ""

In [8]:
# find the closest hotel to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    try:
        #print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        #types_df.loc[index, 'name'] = results[0]['name']
        #types_df.loc[index, 'address'] = results[0]['vicinity']
        #types_df.loc[index, 'price_level'] = results[0]['price_level']
        #types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print(f"No Hotel nearby {row['City']}... skipping")

No Hotel nearby Dzhebariki-Khaya... skipping


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[['Lat', 'Lng']]
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))